In [4]:
from pathlib import Path

base_path = Path.cwd()  # 현재 디렉토리
base_path

PosixPath('/Users/ranifarm/python-work/adaptive-rag-public/unit_test')

In [29]:
import sys
import os

sys.path.append(os.path.abspath("../"))  # utils가 상위 폴더에 있을 경우

from utils.postgresql import keyword_search
from utils.milvus import semantic_search

query = "최저임금"

keyword_json = keyword_search(query, 3)
semantic_json = semantic_search(query, 3)

hybrid_json = keyword_json + semantic_json
len(hybrid_json)

==== condition ==== ::  최저임금
DB 연결 종료!


6

In [30]:
# keyword search result
hybrid_json[0]['metadata']

{'search_type': 'keyword',
 'file_seq': 3,
 'passage_seq': 12,
 'regist_id': 'unknown',
 'regist_dt': '',
 'modify_dt': ''}

In [31]:
# semantic search result
hybrid_json[-1]['metadata']

{'search_type': 'vector',
 'file_seq': 40,
 'passage_seq': 4,
 'regist_id': 'unknown',
 'regist_dt': '',
 'modify_dt': ''}

In [ ]:
# data full field check
hybrid_json[0]

In [32]:
# remove duplicate 
seen = set()
hybrid_new_json = []

for data in hybrid_json:
    field_pair = (data['metadata']['file_seq'], data['metadata']['passage_seq'])
    if field_pair not in seen:
        seen.add(field_pair)
        hybrid_new_json.append(data)

hybrid_json = hybrid_new_json
len(hybrid_json)

6

In [33]:
from sentence_transformers import CrossEncoder

# 모델 로드
reranker = CrossEncoder("BAAI/bge-reranker-v2-m3")

In [34]:
documents = [
    {
        "file_seq": data['metadata']['file_seq'],
        "passage_seq": data['metadata']['passage_seq'],
        "text": f"{data['file_name']}: {data['content']} (Pages: {data['page_numbers']})"
    }
    for data in hybrid_json
]

pairs = [[query, doc['text']] for doc in documents]

len(pairs)

6

In [35]:
scores = reranker.predict(pairs)
scores

array([0.1284332 , 0.14767548, 0.2422166 , 0.9733666 , 0.9736105 ,
       0.9412051 ], dtype=float32)

In [36]:
# file_seq 와 passage_seq 둘 다 비교해서 판별해야 함.
id_to_score = {f"{doc['file_seq']}_{doc['passage_seq']}": score for doc, score in zip(documents, scores)}
id_to_score

{'3_12': 0.1284332,
 '87_19': 0.14767548,
 '108_96': 0.2422166,
 '40_3': 0.9733666,
 '40_0': 0.9736105,
 '40_4': 0.9412051}

In [ ]:
# 기존 스코어는 메타데이터에 넣고 새로운 스코어는 리랭크 점수로 업데이트.
for hdata in hybrid_json:
    hdata["metadata"]["prev_score"] = hdata["score"]
    fseq = hdata["metadata"]["file_seq"]
    pseq = hdata["metadata"]["passage_seq"]
    hdata["score"] = id_to_score[f"{fseq}_{pseq}"]


hybrid_json

In [38]:
hybrid_json.sort(key=lambda x:x['score'], reverse=True)

In [ ]:
# 완료!!
hybrid_json[:4]